# Une fonction pour simplifier l'accès aux données

In [33]:
object FieldUtils extends Serializable {
    def extractField(s: String, fieldNumber: Int): String = {
        val fields = s.split('\t')
        if (fieldNumber >= fields.length) "" else fields(fieldNumber)
    }
}

In [34]:
println(FieldUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 0))
println(FieldUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 1))
println(FieldUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 2))
println(FieldUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 3))
println(FieldUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 4))
println(FieldUtils.extractField("2	CASSIOPEE	2009	33	3.0000", 5))

2
CASSIOPEE
2009
33
3.0000


# Charger les données
1. Créer le RDD `lignes` à partir du répertoire `prenoms_sample.txt`

In [35]:
val lignes = sc.textFile("prenoms_sample.txt")
lignes.take(10).foreach(println)

1	AARON	2011	17	13.0000
1	AARON	2015	33	49.0000
1	ABDALLAH	1977	42	4.0000
1	ABDALLAH	2005	92	13.0000
1	ABDELAZIZ	1971	28	3.0000
1	ABDELAZIZ	1978	08	3.0000
1	ABDERRAHMAN	2011	06	3.0000
1	ABDOULAYE	2005	75	23.0000
1	ABEL	1908	58	8.0000
1	ADAM	2002	22	3.0000


# Transformer les lignes en prénoms
1. En appliquant la méthode `map`, créer le RDD prenoms à partir de `lignes`

In [36]:
val prenoms = lignes.map(l => (
    FieldUtils.extractField(l, 0).charAt(0),
    FieldUtils.extractField(l, 1),
    FieldUtils.extractField(l, 2).toInt,
    FieldUtils.extractField(l, 3).toInt,
    FieldUtils.extractField(l, 4).toDouble.toInt
))
prenoms.take(10).foreach(println)

(1,AARON,2011,17,13)
(1,AARON,2015,33,49)
(1,ABDALLAH,1977,42,4)
(1,ABDALLAH,2005,92,13)
(1,ABDELAZIZ,1971,28,3)
(1,ABDELAZIZ,1978,8,3)
(1,ABDERRAHMAN,2011,6,3)
(1,ABDOULAYE,2005,75,23)
(1,ABEL,1908,58,8)
(1,ADAM,2002,22,3)


# Interroger les données
La documentation des méthodes d'un RDD est disponible ([RDD](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD), [PairRDDFunctions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions)).

1. Rappeler ce que sont les *transformations* et les *actions*
1. Donner, pour chaque prénom, son nombre d'occurences (`map` et `reduceByKey`)

In [37]:
val effectifParPrenoms = prenoms.map(p => (p._2, p._5)).reduceByKey((n1, n2) => n1 + n2)
effectifParPrenoms.take(10).foreach(println)

(BRICE,16)
(OCEANE,17)
(ASSIA,39)
(YOEN,3)
(NICOLLE,27)
(A�NHOA,3)
(AM�LIE,102)
(DANIELE,14)
(ANNE-SOPHIE,35)
(EMMANUEL,186)


1. Donner le nombre total de naissances avec un prénom féminin (`filter`, `map`, `reduce` ou `sum`)

In [38]:
val effectifFéminin = prenoms.filter(p => p._1 == '2').map(p => p._5).sum
println(effectifFéminin)

39747.0


1. Donner l'effectif maximal et minimal par prénom (`map`, `aggregateByKey`)

In [39]:
val prenomsAvecEffectif = prenoms.map(p => (p._2, (p._5, p._5)))
def minMax(p1: Tuple2[Int, Int], p2: Tuple2[Int, Int]): Tuple2[Int, Int] = (math.min(p1._1, p2._1), math.max(p1._2, p2._2))
val minMaxParPrenoms = prenomsAvecEffectif.aggregateByKey((Integer.MAX_VALUE, Integer.MIN_VALUE))(minMax, minMax)
minMaxParPrenoms.take(10).foreach(println)

(BRICE,(3,9))
(OCEANE,(17,17))
(ASSIA,(6,19))
(YOEN,(3,3))
(NICOLLE,(3,17))
(A�NHOA,(3,3))
(AM�LIE,(4,31))
(DANIELE,(14,14))
(ANNE-SOPHIE,(4,31))
(EMMANUEL,(4,148))


1. Sur le modèle des prénoms, charger les données des départements
1. Donner, pour chaque nom de département, le prénom le plus fréquent depuis l'année 2000

In [48]:
val lignesDept = sc.textFile("dpts.txt").filter(l => l.startsWith("REGION") == false)
val depts = lignesDept.map(l => (
    FieldUtils.extractField(l, 0).toInt,
    FieldUtils.extractField(l, 1),
    FieldUtils.extractField(l, 2),
    FieldUtils.extractField(l, 3).toInt,
    FieldUtils.extractField(l, 4),
    FieldUtils.extractField(l, 5)
))
depts.takeSample(false, 10)

Array((6,976,97608,0,MAYOTTE,Mayotte), (27,90,90010,2,TERRITOIRE DE BELFORT,Territoire de Belfort), (28,76,76540,3,SEINE-MARITIME,Seine-Maritime), (76,09,09122,5,ARIEGE,Ari�ge), (44,67,67482,2,BAS-RHIN,Bas-Rhin), (24,18,18033,2,CHER,Cher), (75,87,87085,3,HAUTE-VIENNE,Haute-Vienne), (32,62,62041,2,PAS-DE-CALAIS,Pas-de-Calais), (2,972,97209,3,MARTINIQUE,Martinique), (93,05,05061,4,HAUTES-ALPES,Hautes-Alpes))

In [80]:
val prenomsParDepts = prenoms.filter(p => p._3 >= 2000).map(p => (p._4, (p._2, p._5)))
val maxParDepts = prenomsParDepts.reduceByKey((p1, p2) => (if (p1._2 < p2._2) p2._1 else p1._1, math.max(p1._2, p2._2)))
val deptsAvecNoms = depts.filter(d => d._2 != "2A" && d._2 != "2B").map(d => (d._2.toInt, d._5))
val resultats = deptsAvecNoms.join(maxParDepts).map(j => (j._2._1, (j._2._2._1, j._2._2._2))).sortByKey()
resultats.collect.foreach(println)

(AIN,(ENZO,29))
(AISNE,(NATHA�L,5))
(ALLIER,(_PRENOMS_RARES,100))
(ALPES-DE-HAUTE-PROVENCE,(LUCAS,8))
(ALPES-MARITIMES,(ISMAEL,8))
(ARDECHE,(ALICE,8))
(ARDENNES,(SALOM�,10))
(ARIEGE,(VALENTIN,5))
(AUBE,(ANTONIN,9))
(AUDE,(LOUIS,21))
(AVEYRON,(ROBIN,13))
(BAS-RHIN,(SARA,16))
(BOUCHES-DU-RHONE,(GABRIEL,170))
(CALVADOS,(SOAN,5))
(CHARENTE,(LAURINE,13))
(CHARENTE-MARITIME,(HUGO,53))
(CHER,(MAXIME,10))
(CORREZE,(LINA,14))
(COTE-D'OR,(MARGOT,18))
(COTES-D'ARMOR,(YOUEN,8))
(CREUSE,(_PRENOMS_RARES,16))
(DEUX-SEVRES,(CARLA,12))
(DORDOGNE,(NINA,7))
(DOUBS,(MANON,85))
(DROME,(LAURINE,13))
(ESSONNE,(YANIS,54))
(EURE,(MAXIME,11))
(EURE-ET-LOIR,(HAMZA,5))
(FINISTERE,(MARION,48))
(GARD,(LOUIS,23))
(GERS,(ALEXIS,8))
(GIRONDE,(LOLA,93))
(HAUT-RHIN,(ELISE,15))
(HAUTE-GARONNE,(TONY,12))
(HAUTE-MARNE,(ARTHUR,9))
(HAUTE-SAONE,(L�A,11))
(HAUTE-SAVOIE,(LOUISE,51))
(HAUTE-VIENNE,(WILLIAM,12))
(HAUTES-ALPES,(_PRENOMS_RARES,33))
(HAUTES-PYRENEES,(CL�MENT,11))
(HAUTS-DE-SEINE,(CLARA,70))
(HERAULT,(VICTOIRE,21))


* Fonctions à explorer
  * transformations : groupByKey, coalesce, repartition
  * actions : collect, count
  * persist/cache
  * shuffle
* Idée de requêtes
  * jointure avec les départements